In [1]:
# Install required packages.
import torch_geometric
from torch_geometric.datasets import Planetoid

In [6]:
dataset = Planetoid(root="tutorial1", name="Cora")

In [8]:
print(dataset)
print("Number of graphs: ", len(dataset))
print("Node features: ", dataset.num_node_features)
print("Edge features: ", dataset.num_edge_features)

Cora()
Number of graphs:  1
Node features:  1433
Edge features:  0


In [18]:
print("Dataset features: ", dataset.num_features)
print("Dataset clases: ", dataset.num_classes)
# print(dataset.edge_index)

Dataset features:  1433
Dataset clases:  7


In [9]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [10]:
print(dataset.data.y[:10])

tensor([3, 4, 4, 0, 3, 2, 0, 3, 3, 2])


In [11]:
print(dataset.data.x)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [12]:
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [13]:
data = dataset[0] # We have only one dataset

In [21]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv = SAGEConv(dataset.num_features,
                             dataset.num_classes,
                             aggr = "max")
    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() and use_cuda_if_available else 'cpu')
print(device)
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

cpu


In [23]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [24]:
best_val_acc = test_acc = 0
for epoch in range(1,100):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7280, Test: 0.7220
Epoch: 020, Val: 0.7280, Test: 0.7220
Epoch: 030, Val: 0.7280, Test: 0.7220
Epoch: 040, Val: 0.7280, Test: 0.7220
Epoch: 050, Val: 0.7280, Test: 0.7220
Epoch: 060, Val: 0.7280, Test: 0.7220
Epoch: 070, Val: 0.7280, Test: 0.7220
Epoch: 080, Val: 0.7280, Test: 0.7220
Epoch: 090, Val: 0.7280, Test: 0.7220
